In [2]:
# unzip input file first

import xml.etree.ElementTree as ET
import requests
import math
import pprint
import time
import numpy as np
import pandas as pd

## Travis County Boundary API

In [5]:
response = requests.get("https://geo.traviscountytx.gov/arcgis/rest/services/TNR/TravisCountyBoundary/MapServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")


In [6]:
boundary = response.json()['features'][0]['geometry']['paths'][0]
pprint.pprint(boundary[0])

[-98.02690146055397, 30.25180859020622]


In [7]:
latitudes = []
longitudes = []

for coord in boundary:
    longitudes.append(coord[0])
    latitudes.append(coord[1])
    
print(len(longitudes))
print(len(latitudes))

213
213


## Region Mapping

In [8]:
LATITUDE_MILES = 69
LONGITUDE_MILES = 54.6

vertical_distance = (max(latitudes) - min(latitudes))*LATITUDE_MILES
horizontal_distance = (max(longitudes) - min(longitudes))*LONGITUDE_MILES

In [9]:
# Assuming we want to make 400 squares, how should we partition the vertically/horizontally
NUM_REGIONS = 400
square_len = (vertical_distance*horizontal_distance/NUM_REGIONS)**0.5

In [10]:
latitude_delta = square_len/LATITUDE_MILES
longitude_delta = square_len/LONGITUDE_MILES
longitude_delta

0.03918197759635523

In [11]:
lat_lon = zip(latitudes, longitudes)

In [12]:
min_lat = min(latitudes)
min_lon = min(longitudes)
max_lat = max(latitudes)
max_lon = max(longitudes)
nrows = math.ceil((max_lat - min_lat) / latitude_delta)
ncols = math.ceil((max_lon - min_lon) / longitude_delta)
def map_point_to_region(latitude, longitude):
    return math.floor((latitude-min_lat)/latitude_delta) * ncols  + math.floor((longitude-min_lon)/longitude_delta)

In [13]:
def get_representative(region_num): # 8 cols per row, # R10 --> row_num = 1, col_num = 2
    row_num = region_num//ncols # Correct
    col_num = region_num - row_num*ncols
    lat = min_lat + row_num * latitude_delta + 0.5*latitude_delta
    lon = min_lon + col_num * longitude_delta + 0.5*longitude_delta
    return [lon, lat]

In [14]:
map_point_to_region(30.084446, -97.702885)

33

In [15]:
get_representative(13)

[-97.6441785574555, 30.038787023039202]

In [16]:
region_coords = [get_representative(i) for i in range(400)]
pprint.pprint(region_coords)

[[-98.15354426620812, 30.038787023039202],
 [-98.11436228861176, 30.038787023039202],
 [-98.0751803110154, 30.038787023039202],
 [-98.03599833341904, 30.038787023039202],
 [-97.99681635582269, 30.038787023039202],
 [-97.95763437822635, 30.038787023039202],
 [-97.91845240062999, 30.038787023039202],
 [-97.87927042303363, 30.038787023039202],
 [-97.84008844543727, 30.038787023039202],
 [-97.80090646784092, 30.038787023039202],
 [-97.76172449024456, 30.038787023039202],
 [-97.7225425126482, 30.038787023039202],
 [-97.68336053505185, 30.038787023039202],
 [-97.6441785574555, 30.038787023039202],
 [-97.60499657985915, 30.038787023039202],
 [-97.56581460226279, 30.038787023039202],
 [-97.52663262466643, 30.038787023039202],
 [-97.48745064707008, 30.038787023039202],
 [-97.44826866947372, 30.038787023039202],
 [-97.40908669187736, 30.038787023039202],
 [-97.369904714281, 30.038787023039202],
 [-98.15354426620812, 30.069791892267624],
 [-98.11436228861176, 30.069791892267624],
 [-98.0751803110

## EMS Stations

In [17]:
# Read in data
ems_data = pd.read_excel("ems_facilities_2020Aug12.xlsx")
print(ems_data.shape)

# Filter data
facility_type = ['Medic Station', 'Demand Medic Station']
ems_data = ems_data[ems_data['FACILITY_TYPE'].isin(facility_type)]
print(ems_data.shape)

# Get coords
ems_longitudes = ems_data['LONGITUDE']
ems_latitudes = ems_data['LATITUDE']

ems_coords = [list(c) for c in zip(ems_longitudes, ems_latitudes)]
pprint.pprint(ems_coords)

(51, 30)
(44, 30)
[[-97.7217247499999, 30.25948737],
 [-97.8602694099999, 30.15636358],
 [-97.7503950699999, 30.24662518],
 [-97.6615917999999, 30.32339149],
 [-97.7482999999999, 30.265327],
 [-97.7259999999999, 30.326725],
 [-97.9295778989999, 30.39848581],
 [-97.7678132899999, 30.22977355],
 [-97.8054399999999, 30.206495],
 [-97.7343149999999, 30.274637],
 [-97.6842324899999, 30.27824902],
 [-97.7264146799999, 30.32672533],
 [-97.7394194699999, 30.26582831],
 [-97.7231080899999, 30.37236405],
 [-97.7588594799999, 30.33455506],
 [-97.9348469179999, 30.302810828],
 [-97.7427673899999, 30.41819953],
 [-97.8535901899999, 30.22782184],
 [-97.7132197299999, 30.2292149],
 [-97.6784999999999, 30.35271081],
 [-97.6895245299999, 30.32525961],
 [-97.7936091899999, 30.17470861],
 [-97.8368195599999, 30.40433212],
 [-97.7725225199999, 30.27734164],
 [-97.6923538799999, 30.38399792],
 [-97.8018875799999, 30.44651026],
 [-97.8239838099999, 30.13933909],
 [-97.8200249299999, 30.27945028],
 [-98.0006

## Hospital Locations

In [3]:
BingMapsKey = "AgmNkDXiPX65nDUv6MUa0gRtHyrrGAMuD5n8mNQfl72ISI6xGZA37clQ0OcP56y-"

# Example
call = requests.get("https://dev.virtualearth.net/REST/v1/LocalSearch/?query=hospital&userLocation=47.602038,-122.333964&key=" + BingMapsKey)
pprint.pprint(call.json())

{'authenticationResultCode': 'ValidCredentials',
 'brandLogoUri': 'http://dev.virtualearth.net/Branding/logo_powered_by.png',
 'copyright': 'Copyright © 2020 Microsoft and its suppliers. All rights '
              'reserved. This API cannot be accessed and the content and any '
              'results may not be used, reproduced or transmitted in any '
              'manner without express written permission from Microsoft '
              'Corporation.',
 'resourceSets': [{'estimatedTotal': 10,
                   'resources': [{'Address': {'addressLine': '1100 9th Ave',
                                              'adminDistrict': 'WA',
                                              'countryRegion': 'US',
                                              'formattedAddress': '1100 9th '
                                                                  'Ave, '
                                                                  'Seattle, '
                                                        

In [4]:
query = "hospital"

# Need to negate second coordinate
area = "30.2672,97.7431"
#29.7604,-95.3698

#call = requests.get("https://dev.virtualearth.net/REST/v1/LocalSearch/?query=" + query + "&userLocation=" + area + "&key=" + BingMapsKey)
#call = requests.get("https://dev.virtualearth.net/REST/v1/LocalSearch/?query=coffee&userLocation=30.2672,97.7431&key=" + BingMapsKey)

call = requests.get("https://dev.virtualearth.net/REST/v1/LocalSearch/?type=Hospitals&userLocation=30.2672,-97.7431&maxResults=25&key=" + BingMapsKey)
#call = requests.get("https://dev.virtualearth.net/REST/v1/LocalSearch/?query=hospital&userLocation=30.2672,-97.7431&maxResults=25&key=" + BingMapsKey)

pprint.pprint(call.json())

{'authenticationResultCode': 'ValidCredentials',
 'brandLogoUri': 'http://dev.virtualearth.net/Branding/logo_powered_by.png',
 'copyright': 'Copyright © 2020 Microsoft and its suppliers. All rights '
              'reserved. This API cannot be accessed and the content and any '
              'results may not be used, reproduced or transmitted in any '
              'manner without express written permission from Microsoft '
              'Corporation.',
 'resourceSets': [{'estimatedTotal': 23,
                   'resources': [{'Address': {'addressLine': '4534 W Gate Blvd '
                                                             'Ste 100',
                                              'adminDistrict': 'TX',
                                              'countryRegion': 'US',
                                              'formattedAddress': '4534 W Gate '
                                                                  'Blvd Ste '
                                                   

In [5]:
hospitals = call.json()['resourceSets'][0]['resources']

#pprint.pprint(hospitals)
print(hospitals[1]['point']['coordinates'])
print(hospitals[1]['name'])

[30.246864318847656, -97.77732849121094]
Bluebonnet Animal Hospital


In [6]:
# Extract hosipital info
hospital_coords = []
hospital_names = []

for h in hospitals:
    name = h['name']
    hospital_names.append(name)
    
    coord = h['point']['coordinates']
    flipped_coord = [coord[1], coord[0]]
    hospital_coords.append(flipped_coord)

    
pprint.pprint(hospital_coords)
pprint.pprint(hospital_names)


[[-97.80225372314453, 30.23076629638672],
 [-97.77732849121094, 30.246864318847656],
 [-97.80033111572266, 30.17304229736328],
 [-97.68097686767578, 30.61981201171875],
 [-97.73481750488281, 30.275590896606445],
 [-98.17084503173828, 29.72076416015625],
 [-97.6845932006836, 30.332632064819336],
 [-97.76069641113281, 30.375274658203125],
 [-97.74609375, 30.304731369018555],
 [-97.70735931396484, 30.303455352783203],
 [-97.7254867553711, 30.289552688598633],
 [-97.73509216308594, 30.27420997619629],
 [-97.77456665039062, 30.226062774658203],
 [-97.74533081054688, 30.281185150146484],
 [-97.62997436523438, 30.444177627563477],
 [-97.7745132446289, 30.223644256591797],
 [-97.75154113769531, 30.278207778930664],
 [-97.36383819580078, 31.07794952392578],
 [-97.70702362060547, 30.305355072021484],
 [-97.70703125, 30.305356979370117],
 [-97.79949188232422, 30.21432113647461],
 [-97.73766326904297, 30.272655487060547],
 [-97.94619750976562, 29.567913055419922]]
['Westgate Pet & Bird Hospital',


In [8]:
# Filter hospitals by cleaning search results

i = len(hospital_names) - 1

while(i >= 0):
    name =  hospital_names[i]
    
    # Remove animal hospitals
    if("Animal" in name or "Veterinary" in name or "Pet" in name):
        del hospital_names[i]
        del hospital_coords[i]

    # Remove specialized facilities
    if("Education" in name or "Disorder" in name or "Birth" in name):
        del hospital_names[i]
        del hospital_coords[i]

    i -= 1
    
pprint.pprint(hospital_coords)
pprint.pprint(hospital_names)

[[-97.73481750488281, 30.275590896606445],
 [-98.17084503173828, 29.72076416015625],
 [-97.74609375, 30.304731369018555],
 [-97.70735931396484, 30.303455352783203],
 [-97.7254867553711, 30.289552688598633],
 [-97.73509216308594, 30.27420997619629],
 [-97.77456665039062, 30.226062774658203],
 [-97.7745132446289, 30.223644256591797],
 [-97.36383819580078, 31.07794952392578],
 [-97.70702362060547, 30.305355072021484],
 [-97.70703125, 30.305356979370117],
 [-97.73766326904297, 30.272655487060547],
 [-97.94619750976562, 29.567913055419922]]
['Dell Seton Medical Center at The University of Texas',
 'Two Rivers Eye Care',
 'Ascension Seton Medical Center Austin',
 "Dell Children's Medical Center",
 "St. David's Medical Center",
 'Blackstock Family Health Center',
 "St. David's South Austin Medical Center",
 'Austin Regional Clinic: ARC South OB',
 'Baylor Scott & White Medical Center - Temple',
 "Dell Children's Blood & Cancer Center",
 'Strictly Pediatrics Surgery Center',
 'University of Te

In [22]:
# Save results to csv

hospital_locations = np.array(hospital_coords)
dataset = pd.DataFrame({'hospital_names': hospital_names, "longitude": hospital_locations[:,0], "latitude": hospital_locations[:,1]})
print(dataset)

dataset.to_csv('hospitals.csv', index = False)


                                       hospital_names  longitude   latitude
0   Dell Seton Medical Center at The University of... -97.734818  30.275591
1                                 Two Rivers Eye Care -98.170845  29.720764
2               Ascension Seton Medical Center Austin -97.746094  30.304731
3                      Dell Children's Medical Center -97.707359  30.303455
4                          St. David's Medical Center -97.725487  30.289553
5                     Blackstock Family Health Center -97.735092  30.274210
6             St. David's South Austin Medical Center -97.774567  30.226063
7                Austin Regional Clinic: ARC South OB -97.774513  30.223644
8        Baylor Scott & White Medical Center - Temple -97.363838  31.077950
9               Dell Children's Blood & Cancer Center -97.707024  30.305355
10                 Strictly Pediatrics Surgery Center -97.707031  30.305357
11   University of Texas Physicians - Downtown Austin -97.737663  30.272655
12          

## Batch calls of distance matrix

In [23]:
# Prepare data for api call

#locations = region_coords
locations = region_coords + ems_coords + hospital_coords
durations = []

#pprint.pprint(locations)
print(len(locations))

456


In [32]:
batch_size = 7
num_batches = math.ceil(len(locations) / batch_size)
remainder = len(locations) % batch_size

print(num_batches)

66


In [30]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624884518d9d4075408e95e3401165976247',
    'Content-Type': 'application/json; charset=utf-8'
}

In [31]:
for i in range(num_batches):
    print("batch ", i)
    start = i * batch_size
    sources = list(range(start, start + batch_size))
    
    # handle last partial batch
    if(sources[batch_size - 1] >= len(locations)):
        sources = sources[:remainder]
    
    body = {"locations": locations, "sources": sources}
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
    
    duration_batch = call.json()['durations']
    for j in range(len(duration_batch)):
        durations.append(duration_batch[j])
        
    time.sleep(2)
    
pprint.pprint(durations)


batch  0


KeyError: 'durations'

In [62]:
print(len(durations), len(durations[0]))

455 455


In [63]:
np.savetxt("durations3.csv", np.array(durations), delimiter=",", fmt="%s")